In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [38]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.metrics import f1_score,roc_auc_score, roc_curve, auc
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings("ignore")

**Import the CSV Data as Pandas DataFrame**

In [39]:
df = pd.read_csv('data/Telecom_churn.csv')

**Show Top 5 Records**

In [40]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## Final Feature Decisions

Target:
- Churn (binary, imbalanced)

Dropped Features:
- State (high cardinality, overfitting risk)
- All charge columns (linear transforms of minutes → redundancy)

Numerical Features:
- Account length
- Total day minutes
- Total eve minutes
- Total night minutes
- Total intl minutes
- Customer service calls
- Number vmail messages

Binary Features:
- International plan
- Voice mail plan

Categorical Features:
- Area code

Preprocessing:
- Scale numerical features (StandardScaler)
- One-hot encode categorical features
- Binary features passed as-is

Model:
- Logistic Regression
- class_weight = "balanced"

Metrics:
- ROC-AUC
- F1-score


# **Data Preprocessing**

Defined `Churn` as the target variable and removed high-cardinality and redundant charge features to prevent multicollinearity and improve model stability.

In [41]:
target_col = "Churn"

drop_cols = [
    "State",
    "Total day charge",
    "Total eve charge",
    "Total night charge",
    "Total intl charge"
]

df_model = df.drop(columns=drop_cols)

X = df_model.drop(columns=target_col)
y = df_model[target_col]


Separated features into numeric, binary, and categorical groups to enable appropriate preprocessing and encoding during the modeling pipeline.

In [42]:
numeric_features = [
    "Account length",
    "Total day minutes",
    "Total eve minutes",
    "Total night minutes",
    "Total intl minutes",
    "Customer service calls",
    "Number vmail messages"
]

binary_features = [
    "International plan",
    "Voice mail plan"
]

categorical_features = [
    "Area code"
]

Split the data into train and test sets using stratified sampling to preserve the churn class distribution and ensure reliable evaluation.

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

Created a preprocessing pipeline that scales numeric features, one-hot encodes categorical features, and passes binary features through unchanged for consistent model input.

In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

binary_transformer = Pipeline(steps=[
    ("binary_encode", FunctionTransformer(
        lambda x: (x == "Yes").astype(int)
    ))
])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin", binary_transformer, binary_features)
    ]
)

Built a logistic regression pipeline with class-weight balancing, binary feature encoding, and preprocessing to handle numeric scaling and categorical one-hot encoding, then trained it on the training set.

In [45]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

clf.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

Evaluated the trained model on the test set using ROC-AUC to measure probability ranking performance and F1-score to balance precision and recall for the imbalanced churn dataset.

In [46]:
from sklearn.metrics import (
    classification_report,
    recall_score,
    precision_score,
    roc_auc_score,
    f1_score,
    average_precision_score
)

# probabilities
y_proba = clf.predict_proba(X_test)[:, 1]

# labels
y_pred = (y_proba >= 0.3).astype(int)

print(classification_report(y_test, y_pred))

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
pr_auc = average_precision_score(y_test, y_proba)


print("Recall:", recall)
print("Precision:", precision)
print("ROC-AUC:", roc_auc)
print("F1-score:", f1)
print("PR-AUC:", pr_auc)


              precision    recall  f1-score   support

       False       0.98      0.50      0.66       570
        True       0.24      0.93      0.38        97

    accuracy                           0.57       667
   macro avg       0.61      0.72      0.52       667
weighted avg       0.87      0.57      0.62       667

Recall: 0.9278350515463918
Precision: 0.24128686327077747
ROC-AUC: 0.818249231325737
F1-score: 0.3829787234042553
PR-AUC: 0.41482662312151675


## Logistic Regression with SMOTE

Training SMOTE-enhanced logistic regression model to improve minority class recall through synthetic sample generation during training.


In [47]:
from imblearn.pipeline import Pipeline as ImbPipeline

# Define SMOTE
smote = SMOTE(random_state=42)

# Logistic Regression model
model_smote = LogisticRegression(max_iter=1000, random_state=42)

# Build the pipeline: preprocessing + SMOTE + model
clf_smote = ImbPipeline(steps=[
    ("preprocessor", preprocessor),
    ("smote", smote),
    ("model", model_smote)
])

# Fit on training data
clf_smote.fit(X_train, y_train)

# Evaluate
y_pred_smote = clf_smote.predict(X_test)
y_proba_smote = clf_smote.predict_proba(X_test)[:, 1]


y_pred_smote = (y_proba_smote >= 0.3).astype(int)

print(classification_report(y_test, y_pred_smote))

recall_smote = recall_score(y_test, y_pred_smote)
precision_smote = precision_score(y_test, y_pred_smote)
roc_auc_smote = roc_auc_score(y_test, y_proba_smote)
f1_smote = f1_score(y_test, y_pred_smote)
pr_auc_smote = average_precision_score(y_test, y_proba_smote)

print("SMOTE Model Performance:")
print(f"Recall: {recall_smote}")
print(f"Precision: {precision_smote}")
print(f"ROC-AUC: {roc_auc_smote}")
print(f"F1-score: {f1_smote}")
print(f"PR-AUC: {pr_auc_smote}")


              precision    recall  f1-score   support

       False       0.97      0.54      0.69       570
        True       0.25      0.90      0.39        97

    accuracy                           0.59       667
   macro avg       0.61      0.72      0.54       667
weighted avg       0.86      0.59      0.65       667

SMOTE Model Performance:
Recall: 0.8969072164948454
Precision: 0.2492836676217765
ROC-AUC: 0.8148489781153916
F1-score: 0.3901345291479821
PR-AUC: 0.41555300893950514


## Model Comparison: Standard vs SMOTE


In [48]:
import pandas as pd

comparison = pd.DataFrame({
    'Model': ['Standard LR (class_weight)', 'LR + SMOTE'],
    'Recall': [recall, recall_smote],
    'Precision': [precision, precision_smote],
    'ROC-AUC': [roc_auc, roc_auc_smote],
    'F1-Score': [f1, f1_smote],
    'PR-AUC': [pr_auc, pr_auc_smote]
})

print("\n" + "="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)
print(comparison.to_string(index=False))
print("="*60)

# Show improvements
print(f"\nRecall Improvement: {((recall_smote - recall) / recall * 100):.2f}%")
print(f"Precision Improvement: {((precision_smote - precision) / precision * 100):.2f}%")
print(f"\nF1-Score Improvement: {((f1_smote - f1) / f1 * 100):.2f}%")
print(f"ROC-AUC Change: {((roc_auc_smote - roc_auc) / roc_auc * 100):.2f}%")
print(f"PR-AUC Change: {((pr_auc_smote - pr_auc) / pr_auc * 100):.2f}%")



MODEL PERFORMANCE COMPARISON
                     Model   Recall  Precision  ROC-AUC  F1-Score   PR-AUC
Standard LR (class_weight) 0.927835   0.241287 0.818249  0.382979 0.414827
                LR + SMOTE 0.896907   0.249284 0.814849  0.390135 0.415553

Recall Improvement: -3.33%
Precision Improvement: 3.31%

F1-Score Improvement: 1.87%
ROC-AUC Change: -0.42%
PR-AUC Change: 0.18%


# Additional Models: Random Forest, Decision Tree, XGBoost

Training ensemble and gradient boosting models to benchmark against Logistic Regression.


In [49]:
## Random Forest Classifier

# Build Random Forest pipeline
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight="balanced")

clf_rf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", rf_model)
])

clf_rf.fit(X_train, y_train)

y_pred_rf = clf_rf.predict(X_test)
y_proba_rf = clf_rf.predict_proba(X_test)[:, 1]

classification_report(y_test, y_pred_rf)

y_pred_rf = (y_proba_rf >= 0.3).astype(int)

recall_rf = recall_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
f1_rf = f1_score(y_test, y_pred_rf)
pr_auc_rf = average_precision_score(y_test, y_proba_rf)

print("Random Forest Performance:")
print(f"Recall: {recall_rf}")
print(f"Precision: {precision_rf}")
print(f"ROC-AUC: {roc_auc_rf}")
print(f"F1-score: {f1_rf}")
print(f"PR-AUC: {pr_auc_rf}")


Random Forest Performance:
Recall: 0.7731958762886598
Precision: 0.528169014084507
ROC-AUC: 0.875854584915898
F1-score: 0.6276150627615062
PR-AUC: 0.7428506827871886


In [50]:
## Decision Tree Classifier

dt_model = DecisionTreeClassifier(max_depth=10, random_state=42, class_weight="balanced")

clf_dt = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", dt_model)
])

clf_dt.fit(X_train, y_train)

y_pred_dt = clf_dt.predict(X_test)
y_proba_dt = clf_dt.predict_proba(X_test)[:, 1]

y_pred_dt = (y_proba_dt >= 0.3).astype(int)
classification_report(y_test, y_pred_dt)

recall_dt = recall_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
roc_auc_dt = roc_auc_score(y_test, y_proba_dt)
f1_dt = f1_score(y_test, y_pred_dt)
pr_auc_dt = average_precision_score(y_test, y_proba_dt)

print("Decision Tree Performance:")
print(f"Recall: {recall_dt}")
print(f"Precision: {precision_dt}")
print(f"ROC-AUC: {roc_auc_dt}")
print(f"F1-score: {f1_dt}")
print(f"PR-AUC: {pr_auc_dt}")


Decision Tree Performance:
Recall: 0.6804123711340206
Precision: 0.5365853658536586
ROC-AUC: 0.7795261349249412
F1-score: 0.6
PR-AUC: 0.5569991639819507


In [51]:
## XGBoost Classifier

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=7,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=len(y_train[y_train == False]) / len(y_train[y_train == True]),
    eval_metric='aucpr'
)

clf_xgb = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", xgb_model)
])

clf_xgb.fit(X_train, y_train)
y_proba_xgb = clf_xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_proba_xgb >= 0.3).astype(int)
classification_report(y_test, y_pred_xgb)

recall_xgb = recall_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, y_proba_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
pr_auc_xgb = average_precision_score(y_test, y_proba_xgb)

print("XGBoost Performance:")
print(f"Recall: {recall_xgb}")
print(f"Precision: {precision_xgb}")
print(f"ROC-AUC: {roc_auc_xgb}")
print(f"F1-score: {f1_xgb}")
print(f"PR-AUC: {pr_auc_xgb}")


XGBoost Performance:
Recall: 0.7525773195876289
Precision: 0.6083333333333333
ROC-AUC: 0.8752215590522698
F1-score: 0.6728110599078341
PR-AUC: 0.7758870906764274


## All Models Comparison


In [52]:
all_models_comparison = pd.DataFrame({
    'Model': [
        'Standard LR',
        'LR + SMOTE',
        'Random Forest',
        'Decision Tree',
        'XGBoost'
    ],
    'precision': [precision, precision_smote, precision_rf, precision_dt, precision_xgb],
    'Recall': [recall, recall_smote, recall_rf, recall_dt, recall_xgb],
    'ROC-AUC': [roc_auc, roc_auc_smote, roc_auc_rf, roc_auc_dt, roc_auc_xgb],
    'F1-Score': [f1, f1_smote, f1_rf, f1_dt, f1_xgb],
    'PR-AUC': [pr_auc, pr_auc_smote, pr_auc_rf, pr_auc_dt, pr_auc_xgb]
})

# Sort by Recall descending
all_models_comparison = all_models_comparison.sort_values('Recall', ascending=False).reset_index(drop=True)

print("\n" + "="*70)
print("ALL MODELS PERFORMANCE COMPARISON")
print("="*70)
print(all_models_comparison.to_string(index=False))
print("="*70)

# Highlight best model
best_recall = all_models_comparison.loc[0, 'Model']
best_f1 = all_models_comparison.loc[all_models_comparison['F1-Score'].idxmax(), 'Model']
best_pr_auc = all_models_comparison.loc[all_models_comparison['PR-AUC'].idxmax(), 'Model']

print(f"\n✓ Best Recall: {best_recall}")
print(f"✓ Best F1-Score: {best_f1}")
print(f"✓ Best PR-AUC: {best_pr_auc}")



ALL MODELS PERFORMANCE COMPARISON
        Model  precision   Recall  ROC-AUC  F1-Score   PR-AUC
  Standard LR   0.241287 0.927835 0.818249  0.382979 0.414827
   LR + SMOTE   0.249284 0.896907 0.814849  0.390135 0.415553
Random Forest   0.528169 0.773196 0.875855  0.627615 0.742851
      XGBoost   0.608333 0.752577 0.875222  0.672811 0.775887
Decision Tree   0.536585 0.680412 0.779526  0.600000 0.556999

✓ Best Recall: Standard LR
✓ Best F1-Score: XGBoost
✓ Best PR-AUC: XGBoost


In [53]:
## Best Model Detailed Analysis

# Get best model by F1-Score
best_model_name = all_models_comparison.loc[all_models_comparison['F1-Score'].idxmax(), 'Model']

if best_model_name == 'Standard LR':
    best_model = clf
    y_pred_best = y_pred
elif best_model_name == 'LR + SMOTE':
    best_model = clf_smote
    y_pred_best = y_pred_smote
elif best_model_name == 'Random Forest':
    best_model = clf_rf
    y_pred_best = y_pred_rf
elif best_model_name == 'Decision Tree':
    best_model = clf_dt
    y_pred_best = y_pred_dt
else:  # XGBoost
    best_model = clf_xgb
    y_pred_best = y_pred_xgb

print("\n" + "="*70)
print(f"BEST MODEL ANALYSIS: {best_model_name}")
print("="*70)

cm_best = confusion_matrix(y_test, y_pred_best)
print("\nConfusion Matrix:")
print(cm_best)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

# Show business metrics
tn, fp, fn, tp = cm_best.ravel()
print(f"\nBusiness Metrics:")
print(f"  True Negatives (Correct Non-Churn): {tn}")
print(f"  False Positives (Incorrect Churn): {fp}")
print(f"  False Negatives (Missed Churners): {fn}")
print(f"  True Positives (Correct Churn): {tp}")
print(f"\n  Retention Rate: {(tp / (tp + fn) * 100):.2f}% (Catching churners)")



BEST MODEL ANALYSIS: XGBoost

Confusion Matrix:
[[523  47]
 [ 24  73]]

Classification Report:
              precision    recall  f1-score   support

       False       0.96      0.92      0.94       570
        True       0.61      0.75      0.67        97

    accuracy                           0.89       667
   macro avg       0.78      0.84      0.80       667
weighted avg       0.91      0.89      0.90       667


Business Metrics:
  True Negatives (Correct Non-Churn): 523
  False Positives (Incorrect Churn): 47
  False Negatives (Missed Churners): 24
  True Positives (Correct Churn): 73

  Retention Rate: 75.26% (Catching churners)
